In [ ]:
! pip install hyperopt

In [10]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [11]:
## load the dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [12]:
# splitting the dataset

train,test = train_test_split(data,test_size=0.25,random_state=42)

In [29]:
#splitting the input and output(dependent and independent features)
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel() # .ravel gives 1 D array of independent(output) features

## test dataset
test_x = test.drop(['quality'],axis=1).values
test_y = test[['quality']].values.ravel()


## Further splitting the train data into train and validation
train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

signature = infer_signature(train_x,train_y)


In [30]:
## ANN Model

def train_model(params,epochs,train_x,train_y,test_x,test_y):

    # defining the model architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    # train the ANN model with lr and momentum parameters and track each of them with mlflow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y)
                  ,epochs=epochs,
                  batch_size=32)
        
        # Evaluate the model
        eval_results = model.evaluate(valid_x,valid_y,batch_size=32)
        
        eval_rmse = eval_results[1]

        ## log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        #log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}
    
    


In [34]:
# taken from hyperopt documentation

def objective(params):
    # MLFLOW will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        # valid_x=valid_x,
        # valid_y =valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [35]:
# using hyperopt to set the params
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)), #setting the lr from 10^-5 to 10^-1
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [36]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # log the best parameters,loss and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run['loss'])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)

    #Print out the best parameters and corresponding loss
    print(f"Best parameters:{best}")
    print(f"Best eval rmse:{best_run['loss']}")


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:From c:\Users\VS70\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.

Epoch 1/3                                            

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:From c:\Users\VS70\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.

92/92 [==============================] - 1s 4ms/step - loss: 22.2667 - root_mean_squared_error: 4.7188 - val_loss: 20.7118 - val_root_mean_squared_error: 4.5510

Epoch 2/3                                            

92/92 [==============================] - 0s 2ms/step - loss: 19.0558 - root_mean_squared_error: 4.3653 - val_loss: 17.6952 - v

INFO:tensorflow:Assets written to: C:\Users\VS70\AppData\Local\Temp\tmpkb7hym40\model\data\model\assets



Epoch 1/3                                                                     

92/92 [==============================] - 1s 3ms/step - loss: 11.2295 - root_mean_squared_error: 3.3510 - val_loss: 3.4815 - val_root_mean_squared_error: 1.8659

Epoch 2/3                                                                     

92/92 [==============================] - 0s 2ms/step - loss: 2.6428 - root_mean_squared_error: 1.6257 - val_loss: 1.5322 - val_root_mean_squared_error: 1.2378

Epoch 3/3                                                                     

92/92 [==============================] - 0s 1ms/step - loss: 1.0721 - root_mean_squared_error: 1.0354 - val_loss: 0.9366 - val_root_mean_squared_error: 0.9678

23/23 [==============================] - 0s 924us/step - loss: 0.9366 - root_mean_squared_error: 0.9678

 25%|██▌       | 1/4 [00:27<01:17, 25.89s/trial, best loss: 3.888934373855591]INFO:tensorflow:Assets written to: C:\Users\VS70\AppData\Local\Temp\tmp3osv9d_n\model\data\model

INFO:tensorflow:Assets written to: C:\Users\VS70\AppData\Local\Temp\tmp3osv9d_n\model\data\model\assets



Epoch 1/3                                                                      

92/92 [==============================] - 1s 3ms/step - loss: 17.2001 - root_mean_squared_error: 4.1473 - val_loss: 5.6522 - val_root_mean_squared_error: 2.3774

Epoch 2/3                                                                      

92/92 [==============================] - 0s 2ms/step - loss: 3.3828 - root_mean_squared_error: 1.8393 - val_loss: 2.3220 - val_root_mean_squared_error: 1.5238

Epoch 3/3                                                                      

92/92 [==============================] - 0s 1ms/step - loss: 2.0321 - root_mean_squared_error: 1.4255 - val_loss: 1.8973 - val_root_mean_squared_error: 1.3774

23/23 [==============================] - 0s 1ms/step - loss: 1.8973 - root_mean_squared_error: 1.3774

 50%|█████     | 2/4 [00:37<00:33, 16.97s/trial, best loss: 0.9677677750587463]INFO:tensorflow:Assets written to: C:\Users\VS70\AppData\Local\Temp\tmp457xwxxv\model\data\mod

...

Epoch 1/3                                                                      

92/92 [==============================] - 1s 4ms/step - loss: 16.7523 - root_mean_squared_error: 4.0930 - val_loss: 5.2176 - val_root_mean_squared_error: 2.2842

Epoch 2/3                                                                      

92/92 [==============================] - 0s 2ms/step - loss: 3.3282 - root_mean_squared_error: 1.8243 - val_loss: 2.5507 - val_root_mean_squared_error: 1.5971

Epoch 3/3                                                                      

92/92 [==============================] - 0s 2ms/step - loss: 2.1543 - root_mean_squared_error: 1.4678 - val_loss: 2.0230 - val_root_mean_squared_error: 1.4223

23/23 [==============================] - 0s 1ms/step - loss: 2.0230 - root_mean_squared_error: 1.4223

 75%|███████▌  | 3/4 [00:48<00:14, 14.29s/trial, best loss: 0.9677677750587463]INFO:tensorflow:Assets written to: C:\Users\VS70\AppData\Local\Temp\tmp4_o09h2m\model\data\mod

INFO:tensorflow:Assets written to: C:\Users\VS70\AppData\Local\Temp\tmp4_o09h2m\model\data\model\assets



100%|██████████| 4/4 [00:57<00:00, 14.50s/trial, best loss: 0.9677677750587463]
INFO:tensorflow:Assets written to: C:\Users\VS70\AppData\Local\Temp\tmpmj_z1_da\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\VS70\AppData\Local\Temp\tmpmj_z1_da\model\data\model\assets



Best parameters:{'lr': 0.0002418650362141726, 'momentum': 0.9810940966289233}
Best eval rmse:0.9677677750587463
